In [176]:
from pwn import *

def recv_options():
    for _ in range(4):
        r.recvline()

def send_opt_1(msg):
    r.sendline(b"1")
    r.recvline() # Who are you?
    r.sendline(msg.encode())
    sig_r, sig_s = r.recvline().decode()[7:-2].split(', ')
    return int(sig_r), int(sig_s)
        
r = remote('edu-ctf.csie.org', 42072)
Px, Py = r.recvline().decode()[5:-2].split(', ')

[x] Opening connection to edu-ctf.csie.org on port 42072
[x] Opening connection to edu-ctf.csie.org on port 42072: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42072: Done


In [177]:
Px, Py

('64392810160624940508847684630410816606406861610104484312427348073124019785884',
 '66338332133599813590038292287049972698784107746012848593058891691364696166039')

In [178]:
from ecdsa import SECP256k1
from ecdsa import ellipticcurve
from ecdsa.ecdsa import Public_key, Private_key, Signature

E = SECP256k1
G, n = E.generator, E.order
pubkey = Public_key(G, ellipticcurve.PointJacobi(E, Px, Py, 1, n))

In [179]:
recv_options()
r1, s1 = send_opt_1('123')
recv_options()
r2, s2 = send_opt_1('456')

In [180]:
(r1, s1), (r2, s2)

((112882173018052603293745941111953529824014906307946459772377713391922405655824,
  99940072238367737792514479509760731582355159583249584125181258635240204782777),
 (95866935456207827771873264507777020462265354621865128404736076181932158933462,
  61562504573408324317406178162427821826441489037492565713919037733468767084014))

In [181]:
#s1k = h1 + dr1
#s2k*1337 = h2 + dr2
#s1/(s2*1337) = (h1+dr1)/(h2+dr2)
#s1h2 + d(s1r2) = s2h1*1337 + d(s2r1*1337)

from hashlib import sha256
from Crypto.Util.number import *

h1 = bytes_to_long(sha256('123'.encode()).digest())
h2 = bytes_to_long(sha256('456'.encode()).digest())

d = (s1*h2 - (s2*h1*1337)%n) * inverse_mod((s2*r1*1337)%n - s1*r2, n) % n
d

6970365145873492024248882479220790963874925614365751486748777127652387062245

In [182]:
k = ((h1 + d * r1) % n * inverse_mod(s1, n)) % n
(k * s1) % n == (h1 + d * r1) % n

True

In [183]:
Public_key(G, d*G).point.x(), Px

(mpz(64392810160624940508847684630410816606406861610104484312427348073124019785884),
 '64392810160624940508847684630410816606406861610104484312427348073124019785884')

In [184]:
k = k * 1337 % n
k = k * 1337 % n
prikey = Private_key(pubkey, d)
sig = prikey.sign(bytes_to_long(sha256('Kuruwa'.encode()).digest()), k)
str(sig.r), str(sig.s)

('53334204283711536993134427079146434996152315415453964071687627791422886380164',
 '54269016164161175696975715796188342051628521190376357854373530355722884030645')

In [185]:
r.recvline()
r.recvline()
r.recvline()
r.recvline()

b'3) exit\n'

In [186]:
r.sendline(b"2")
r.sendline(b"Kuruwa")
r.sendline(str(sig.r).encode())
r.sendline(str(sig.s).encode())

In [187]:
r.recvline()

b'username: r: s: FLAG{fcf8ab2bc7b42bbd00e5be2b3d311ec6e8a89526}\n'